In [2]:
import pandas as pd
from scapy.all import *
import matplotlib.pyplot as plt
import seaborn as sns

def extract_jitter_info(pcap_file):
    packets = rdpcap(pcap_file)
    timestamps = [pkt.time for pkt in packets if pkt.haslayer(UDP)]
    jitter_values = []

    for i in range(1, len(timestamps)):
        x = abs(timestamps[i] - timestamps[i - 1])
        jitter_values.append(x if x<0.6 else 0)

    return jitter_values

# Group files based on with or without policies and fast/faster/flood subgroups
with_files = {
    'fast': ['with_tcpdump_output_fast_1.pcap',
             'with_tcpdump_output_fast_2.pcap',
             'with_tcpdump_output_fast_3.pcap'],
    'faster': ['with_tcpdump_output_faster_1.pcap',
               'with_tcpdump_output_faster_2.pcap',
               'with_tcpdump_output_faster_3.pcap'],
    'flood': ['with_tcpdump_output_flood_1.pcap',
              'with_tcpdump_output_flood_2.pcap',
              'with_tcpdump_output_flood_3.pcap']
}

without_files = {
    'fast': ['without_tcpdump_output_fast_1.pcap',
             'without_tcpdump_output_fast_2.pcap',
             'without_tcpdump_output_fast_3.pcap'],
    'faster': ['without_tcpdump_output_faster_1.pcap',
               'without_tcpdump_output_faster_2.pcap',
               'without_tcpdump_output_faster_3.pcap'],
    'flood': ['without_tcpdump_output_flood_1.pcap',
              'without_tcpdump_output_flood_2.pcap',
              'without_tcpdump_output_flood_3.pcap']
}

# Extract jitter values from each file and calculate median for each subgroup
with_data = {}
without_data = {}

for subgroup, files in with_files.items():
    with_data[subgroup] = []
    for file in files:
        with_data[subgroup].extend(extract_jitter_info(file))
    with_data[subgroup] = pd.DataFrame(with_data[subgroup], columns=[subgroup]).median(axis=1)

for subgroup, files in without_files.items():
    without_data[subgroup] = []
    for file in files:
        without_data[subgroup].extend(extract_jitter_info(file))
    without_data[subgroup] = pd.DataFrame(without_data[subgroup], columns=[subgroup]).median(axis=1)

# Plot median jitter values for each group and subgroup
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

sns.set_palette('Set2')

for i, (group, data) in enumerate([('With policies', with_data), ('Without policies', without_data)]):
    for j, (subgroup, median) in enumerate(data.items()):
        median.plot(ax=axes[i, j], kind='line', color='C{}'.format(j), linewidth=2)
        axes[i, j].set_title(group+"_"+subgroup)
        axes[i, j].set_xlabel('Time Index')
        axes[i, j].set_ylabel('Median jitter (seconds)')
    axes[i, 0].set_ylabel
plt.savefig("jitter.pdf")

KeyboardInterrupt: 